# Convolutional Layer

- toc: true
- badges: true
- comments: true
- categories: [jupyter]
- image: images/chart-preview.png
- hide: true

## Introduction


## Let's Start

In [154]:
import jax 
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time 

from typing import Tuple, List, Any, Dict, Callable

### Convolutional Layer

The first function argument is the number of filters and the second argument is the shape of the kernel.  There are other parameters too, but I'm going to keep things simple for now.

In [571]:
conv_layer = tf.keras.layers.Conv2D(filters=2, kernel_size=(2, 2))

Arrays fed to `layer` must be a 4D tensor where the first axis is the batch size, the second axis is the width, the third axis is the height, and thr fourth axis is 
the number of input channels.

In [572]:
input_image = np.random.randn(4,4,3)

Because Keras layers operate on batches of data, and not individual examples, a batch dimension must be addes to `input_image`.  To do this in numpy, take `input_image` and add a `None` in the axis you want to add:

In [573]:
input_batch = input_image[None,:,:,:]

In [574]:
feature_maps = conv_layer(input_batch)

In [575]:
feature_maps.shape

TensorShape([1, 3, 3, 2])

In [576]:
kernels, biases = conv_layer.get_weights()

In [578]:
print(f'kernels shape = {kernels.shape}, biases shape ={biases.shape}')

kernels shape = (2, 2, 3, 2), biases shape =(2,)


In [582]:
def conv2d_(x, kernel):
    y = np.zeros((3,3))
    
    y[0,0] = np.sum(kernel * x[0:2,0:2])
    y[0,1] = np.sum(kernel * x[0:2,1:3])
    y[0,2] = np.sum(kernel * x[0:2,2:4])
    y[1,0] = np.sum(kernel * x[1:3,0:2])
    y[1,1] = np.sum(kernel * x[1:3,1:3])
    y[1,2] = np.sum(kernel * x[1:3,2:4])
    y[2,0] = np.sum(kernel * x[2:4,0:2])
    y[2,1] = np.sum(kernel * x[2:4,1:3])
    y[2,2] = np.sum(kernel * x[2:4,2:4])    
    
    return y

In [585]:
def conv2d(x, kernel):
    xm, xn = x.shape 
    km, kn = kernel.shape 
    
    y = np.zeros((xm - km + 1, xn - kn + 1))
    
    ym, yn = y.shape
    for i in range(ym):
        for j in range(yn):
            y[i, j] = np.sum(kernel * x[i:i+km, j:j+kn]) 
    
    return y

Here's a faily inefficient way to duplicate the evaluation of the `conv_layer` defined in above.  

In [590]:
def convolve(input_image, kernels, biases):
    
    width, height, input_chans = input_image.shape 
    km, kn, _, output_chans = kernels.shape
    
    w = width - km + 1
    h = height - kn + 1 
    
    features = np.zeros((w, h, output_chans))

    for out_chan in range(output_chans):
        y = np.zeros((w, h))
        for in_chan in range(input_chans):
            y += conv2d(
                input_image[:,:,in_chan],
                kernels[:,:,in_chan, out_chan], 
            )
        
        # one bias per output feature
        features[:,:,out_chan] = y + biases[out_chan]
    
    return features

In [591]:
features_ = convolve(input_image, kernels, biases)[None,:,:,:]

In [592]:
assert np.all(np.isclose(feature_maps, features_))

## Max Pooling

In [593]:
pooling_layer = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2))

In [594]:
yy = pooling_layer(feature_maps)
print(f'{feature_maps.shape} -> {yy.shape}')

(1, 3, 3, 2) -> (1, 1, 1, 2)


In [540]:
yy

<tf.Tensor: shape=(1, 1, 1, 2), dtype=float32, numpy=array([[[[0.9654248, 1.2988867]]]], dtype=float32)>

In [274]:
print(feature_maps)

tf.Tensor(
[[[[ 0.89900464 -1.0616233 ]
   [-0.10247962  1.2988867 ]
   [-0.8515937   0.6712394 ]]

  [[ 0.9654248   0.23607667]
   [-0.43404913  0.27193436]
   [-0.5712364  -0.73882663]]

  [[ 0.6847748  -0.8419535 ]
   [ 0.46285468  0.7197448 ]
   [-1.3353215   0.47066653]]]], shape=(1, 3, 3, 2), dtype=float32)


In [275]:
print(yy)

tf.Tensor(
[[[[ 0.9654248   1.2988867 ]
   [-0.10247962  1.2988867 ]]

  [[ 0.9654248   0.7197448 ]
   [ 0.46285468  0.7197448 ]]]], shape=(1, 2, 2, 2), dtype=float32)


In [279]:
print(feature_maps[0,:,:,1])

tf.Tensor(
[[-1.0616233   1.2988867   0.6712394 ]
 [ 0.23607667  0.27193436 -0.73882663]
 [-0.8419535   0.7197448   0.47066653]], shape=(3, 3), dtype=float32)


In [280]:
yy[0,:,:,1]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1.2988867, 1.2988867],
       [0.7197448, 0.7197448]], dtype=float32)>

In [304]:
input_batch.shape

(1, 4, 4, 3)

In [365]:
list(range(0,10,2))

[0, 2, 4, 6, 8]

In [563]:
def pool2D(x, pool_size=(2,2), strides=(1,1), fn=np.max):
    xm, xn = x.shape 
    pm, pn = pool_size 
    sm, sn = strides
    
    ym, yn = (xm-pm+1) // sm, (xn-pn+1) // sn

    y = np.zeros((ym, yn))
    
    ii = 0
    for i in range(0, xm-pm+1, sm):
        jj = 0
        for j in range(0, xn-pn+1, sn):
            y[ii,jj] = fn(x[i:i+pm,j:j+pn])
            jj += 1
        ii += 1
    return y

In [595]:
#def pool2D_(x, pool_size=(2,2), strides=(1,1)):
##    xm, xn = x.shape 
#    pm, pn = pool_size 
#    sm, sn = strides
#    
#    y = np.zeros((xm - pm + 1, xn - pn + 1))
#    
#    ym, yn = y.shape
#    
#    for i in range(0, ym):
#        for j in range(0, yn):
#            y[i,j] = np.max(x[i:i+pm,j:j+pn])
#    
#    return y

In [596]:
x = np.random.randn(3,3)
b = pool2D(x, strides=(2,2))
print(a)
print(b)
print(x)

[[-0.36715236  0.09317793]
 [-0.36715236 -0.06301992]]
[[0.98810509]]
[[-0.92430295 -0.55469247 -0.59627284]
 [ 0.98810509 -0.64934654  0.29590853]
 [ 1.21270553  0.98248372 -0.38071894]]


In [597]:
def pooling(features, pool_size=(2,2), strides=(2,2)):
    
    px, py = pool_size
    sm, sn = strides
    width, height, chans = features.shape 
    
    m, n = (width - px + 1) // sm, (height - py + 1) // sn
    
    features_ = np.zeros((m, n, chans))

    # Note that we're not changing the number of features
    for chan in range(chans):
        features_[:,:,chan] = pool2D(features[:,:,chan], pool_size, strides)
    
    return features_

In [598]:
pooling(feature_maps[0,:,:,:])

array([[[ 1.3011235 , -0.24245605]]])